## Homomorphic Encryption Basic Working

In [69]:
from mylib import paillier

In [70]:
# Create Public and Private Keys
key_length = 1024
pub_key, private_key = paillier.generate_paillier_keypair(n_length=key_length) 

In [71]:
pub_key

<PaillierPublicKey e7cc58ca1c>

In [72]:
private_key

<PaillierPrivateKey for <PaillierPublicKey e7cc58ca1c>>

In [73]:
# Encrypt an operation using Public Key
a = 10
print("a: ",a)

encrypted_a = pub_key.encrypt(a)
print("Encrypted a: ",encrypted_a)

print("Encrypted a Public Key: ", encrypted_a.public_key)

a:  10
Encrypted a:  <mylib.paillier.EncryptedNumber object at 0x000001FE56435050>
Encrypted a Public Key:  <PaillierPublicKey e7cc58ca1c>


In [74]:
# Encrypt another variable
b = 5
print("b: ", b)

encrypted_b = pub_key.encrypt(b)
print("Encrypted b: ", encrypted_b)

print("Encrypted b Public Key: ",encrypted_b.public_key)

b:  5
Encrypted b:  <mylib.paillier.EncryptedNumber object at 0x000001FE1574E7D0>
Encrypted b Public Key:  <PaillierPublicKey e7cc58ca1c>


In [75]:
# Do an operation on Encrypted Variables
c = a + b
print("c: ", c)

c:  15


In [76]:
d = a * b
print("d: ",d)

d:  50


In [77]:
e = a - b

encrypted_e = pub_key.encrypt(e)
print("Encrypted e: ", encrypted_e)

Encrypted e:  <mylib.paillier.EncryptedNumber object at 0x000001FE56437A10>


In [78]:
# Decrypt the Encrypted Data
decrypted_e = private_key.decrypt(encrypted_e)

In [79]:
print("Decrypted e: ", decrypted_e)

Decrypted e:  5


## Homomorphic Encryption for Machine Learning

Logistic Regression for Spam/Not Spam e-mail Classification.

For this problem we have two users:

**USER-1**

**USER-2**

AI Inc. makes a Machine Learning model that is trained on some email data for classification between Spam/Not Spam. Now, they want to take that model, encrypt it and send to USER-1 and USER-2 who will train the model on their data, fully Homomorphically Encrypted, and send the trained, a bit better model back to AI Inc.

In this process, AI Inc. get a better trained model every time without even looking at USER-1 or USER-2 data. This way AI Inc. can serve the customers better with a smart Machine Learning model and the USER has complete control of his/her data.

In [80]:
# Import Dependencies

import time
import os.path
from zipfile import ZipFile
from urllib.request import urlopen
from contextlib import contextmanager

import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer

In [81]:
# Data Preprocessing
def preprocess_data():
    """
    Load the email dataset and Represent them as bag-of-words.
    Shuffle and split train/test.
    """

    print("Importing dataset...")
    path = './dataset/enron1/ham/'
    ham1 = [open(path + f, 'r', errors='replace').read().strip(r"\n")
            for f in os.listdir(path) if os.path.isfile(path + f)]
    path = './dataset/enron1/spam/'
    spam1 = [open(path + f, 'r', errors='replace').read().strip(r"\n")
             for f in os.listdir(path) if os.path.isfile(path + f)]
    path = './dataset/enron2/ham/'
    ham2 = [open(path + f, 'r', errors='replace').read().strip(r"\n")
            for f in os.listdir(path) if os.path.isfile(path + f)]
    path = './dataset/enron2/spam/'
    spam2 = [open(path + f, 'r', errors='replace').read().strip(r"\n")
             for f in os.listdir(path) if os.path.isfile(path + f)]

    # Merge and create labels
    emails = ham1 + spam1 + ham2 + spam2
    y = np.array([-1] * len(ham1) + [1] * len(spam1) +
                 [-1] * len(ham2) + [1] * len(spam2))

    # Words count, keep only frequent words
    # Minimum Document Word Frequency: 0.001
    count_vect = CountVectorizer(decode_error='replace', stop_words='english', min_df=0.001)
    X = count_vect.fit_transform(emails)

    print('Vocabulary size: %d' % X.shape[1])

    # Shuffle
    perm = np.random.permutation(X.shape[0])
    X, y = X[perm, :], y[perm]

    # Split train and test
    split = 500
    X_train, X_test = X[-split:, :], X[:-split, :]
    y_train, y_test = y[-split:], y[:-split]

    print("Labels in trainset are {:.2f} spam : {:.2f} ham".format(
        np.mean(y_train == 1), np.mean(y_train == -1)))

    return X_train, y_train, X_test, y_test

In [82]:
@contextmanager
def timer():
    """Helper for measuring runtime"""

    time0 = time.perf_counter()
    yield
    print('[elapsed time: %.2f s]' % (time.perf_counter() - time0))

In [83]:
class AI_Inc:
    """
    AI Inc. Trains a Logistic Regression model on plaintext data, encrypts the model for remote use by USER-1 and USER-2,
    decrypts encrypted scores using the paillier private key.
    """

    def __init__(self):
        self.model = LogisticRegression()

    # Generate Public and Private Key Pairs
    # Public Key is used to Encrypt the Data, Private Key to Decrypt
    def generate_paillier_keypair(self, n_length):
        self.pubkey, self.privkey = paillier.generate_paillier_keypair(n_length=n_length)

    # Train the Model
    def fit(self, X, y):
        self.model = self.model.fit(X, y)

    # Make Predictions for Email "Spam/Not Spam"
    def predict(self, X):
        return self.model.predict(X)

    # Encypt the Coefficients for the Logistic Regression Equation
    # Weights can tell about the data, so Encrypt them
    # Equation: y = mX + b
    def encrypt_weights(self):
        coef = self.model.coef_[0, :]
        encrypted_weights = [self.pubkey.encrypt(coef[i])
                             for i in range(coef.shape[0])]
        encrypted_intercept = self.pubkey.encrypt(self.model.intercept_[0])
        return encrypted_weights, encrypted_intercept

    # Decrypt the Scores for the Model
    def decrypt_scores(self, encrypted_scores):
        return [self.privkey.decrypt(s) for s in encrypted_scores]

In [84]:
# Now the USER-1 gets a trained model from AI Inc. and trains on its own data all using Homomorphic Encryption.
class User_1:
    """
    USER-1/USER-2 are given the encrypted model trained by AI Inc. and the public key.

    Scores local plaintext data with the encrypted model, but cannot decrypt
    the scores without the private key held by AI Inc..
    """

    def __init__(self, pubkey):
        self.pubkey = pubkey

    # Set Initial Values of Coefficients
    def set_weights(self, weights, intercept):
        self.weights = weights
        self.intercept = intercept

    # Compute the Prediction Scores for the Model all while being totally Encrypted.
    def encrypted_score(self, x):
        """Compute the score of `x` by multiplying with the encrypted model,
        which is a vector of `paillier.EncryptedNumber`"""
        score = self.intercept
        _, idx = x.nonzero()
        for i in idx:
            score += x[0, i] * self.weights[i]
        return score

    # Get the Evaluation Scores for the Model
    def encrypted_evaluate(self, X):
        return [self.encrypted_score(X[i, :]) for i in range(X.shape[0])]

In [85]:
# Get the Preprocessed Split Data
X_train, y_train, X_test, y_test = preprocess_data()

Importing dataset...
Vocabulary size: 7994
Labels in trainset are 0.28 spam : 0.72 ham


In [86]:
# Now firstly the AI Inc. Generates the Public and Private Keys
print("AI Inc.: Generating Paillier Public Private Keypair")
ai_inc = AI_Inc()
# NOTE: using smaller keys sizes wouldn't be cryptographically safe
ai_inc.generate_paillier_keypair(n_length=1024)

AI Inc.: Generating Paillier Public Private Keypair


In [87]:
print("AI Inc.: Training Initial Spam Classifier")
with timer() as t:
    ai_inc.fit(X_train, y_train)

AI Inc.: Training Initial Spam Classifier
[elapsed time: 0.96 s]


In [88]:
print("AI Inc.'s Classification on Test Data, what it would expect the performance to be on USER-1/2's data...")
with timer() as t:
    error = np.mean(ai_inc.predict(X_test) != y_test)
print("Error {:.3f}".format(error))

AI Inc.'s Classification on Test Data, what it would expect the performance to be on USER-1/2's data...
[elapsed time: 0.05 s]
Error 0.043


In [89]:
print("AI Inc.: Encrypting Trained Classifier before sending to USER-1/2")
with timer() as t:
    encrypted_weights, encrypted_intercept = ai_inc.encrypt_weights()

AI Inc.: Encrypting Trained Classifier before sending to USER-1/2
[elapsed time: 105.68 s]


In [90]:
# Confirming the Weights are Encrypted
print("Encrypted Weights: ", encrypted_weights)
print("Encrypted Intercept: ", encrypted_intercept)

Encrypted Weights:  [<mylib.paillier.EncryptedNumber object at 0x000001FE5A015950>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A0168D0>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A027590>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A015450>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A014150>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A016390>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A017510>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A016F10>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A017D10>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A015590>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A015E90>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A017790>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A017310>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A014A90>, <mylib.paillier.EncryptedNumber object at 0x000001FE5A027B10>, <mylib.paillier.EncryptedNumber ob

Now, we have an encrypted trained model.

AI Inc. sends the trained model with it's weights encrypted [as weights can tell something about the data] and sends both the things to the USER-1 and USER2.

Now, USER-1 and USER-2 get the encrypted weights, the trained model and the public key to do some operations on their own dataset. This is called **Homomorphic Encryption**.

In [91]:
# USER-1 taking the encrypted model, weights and testing performance on it's own dataset
print("USER-1: Scoring on own data with AI Inc.'s Encrypted Classifier...")

# AI Inc sends the Public Keys to perform operations
user_1 = User_1(ai_inc.pubkey)

# USER-1 sets the model Hyperparameters to AI Inc.'s Hyperparameter values
user_1.set_weights(encrypted_weights, encrypted_intercept)

with timer() as t:
    encrypted_scores = user_1.encrypted_evaluate(X_test)

USER-1: Scoring on own data with AI Inc.'s Encrypted Classifier...
[elapsed time: 130.10 s]


In [92]:
# Making Sure the Score is Encrypted
print(encrypted_scores)

[<mylib.paillier.EncryptedNumber object at 0x000001FE57560990>, <mylib.paillier.EncryptedNumber object at 0x000001FE56446850>, <mylib.paillier.EncryptedNumber object at 0x000001FE56D5E610>, <mylib.paillier.EncryptedNumber object at 0x000001FE1562D250>, <mylib.paillier.EncryptedNumber object at 0x000001FE564478D0>, <mylib.paillier.EncryptedNumber object at 0x000001FE56446E50>, <mylib.paillier.EncryptedNumber object at 0x000001FE56444D90>, <mylib.paillier.EncryptedNumber object at 0x000001FE56447B50>, <mylib.paillier.EncryptedNumber object at 0x000001FE56BCBE90>, <mylib.paillier.EncryptedNumber object at 0x000001FE56445750>, <mylib.paillier.EncryptedNumber object at 0x000001FE56445650>, <mylib.paillier.EncryptedNumber object at 0x000001FE56444450>, <mylib.paillier.EncryptedNumber object at 0x000001FE55DF1ED0>, <mylib.paillier.EncryptedNumber object at 0x000001FE564479D0>, <mylib.paillier.EncryptedNumber object at 0x000001FE56444810>, <mylib.paillier.EncryptedNumber object at 0x000001FE56

Now USER has the option to train the model on it's own data and send the trained model to AI Inc.

In [93]:
print("AI Inc.: Decrypting USER-1/2's scores")

with timer() as t:
    scores = ai_inc.decrypt_scores(encrypted_scores)
    error = np.mean(np.sign(scores) != y_test)
print("Error {:.3f} -- this is not known to AI Inc., who does not possess the ground truth labels".format(error))

AI Inc.: Decrypting USER-1/2's scores
[elapsed time: 40.91 s]
Error 0.043 -- this is not known to AI Inc., who does not possess the ground truth labels


In [94]:
from sslib import shamir
required_shares = 2
distributed_shares = 5
shamir.to_base64(shamir.split_secret("this is my secret".encode('ascii'), required_shares, distributed_shares))
{'required_shares': 2, 'prime_mod': 'AQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAhQ==', 'shares': ['1-Swwdr0O19NSMsoG4DXxvTeB9WTykw9+a', '2-lhg7XodrvzSw+5BPsYW+LkfaPxPmFVnA', '3-4SRZDcshiZTVRJ7nVY8NDq83JOsnZtPm', '4-LDB2vQ7XU/T5ja1++Zhb7xaUCsJouE2H', '5-dzyUbFKNHlUd1rwWnaGqz33w8JmqCcet']}

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/dev/random'

In [ ]:
from sslib import shamir
data = {'required_shares': 2, 'prime_mod': 'AQAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAhQ==', 'shares': ['1-Swwdr0O19NSMsoG4DXxvTeB9WTykw9+a', '3-4SRZDcshiZTVRJ7nVY8NDq83JOsnZtPm']}
shamir.recover_secret(shamir.from_base64(data)).decode('ascii')
'this is my secret'

'this is my secret'

In [ ]:


import secrets
import numpy as np
import sympy
import itertools


class SSS:
    """
        k = the key
        p = prime number for modulo operations in the set Z_p
        w = number of shares
        t = minimum number of shares required to reconstruct the key k
    """

    def __init__(self, k, p, w, t):
        if not (sympy.isprime(p)):
            raise ValueError('p is not prime')
        else:
            self.k = k
            self.p = p
            self.w = w
            self.t = t

    '''
        SHARES GENERATION
    '''

    '''
        choose_x chooses w distinct, non-zero elements of Z_p
        these are the public x_i values, 1 <= i <= w
    '''

    def choose_x(self):
        x = []
        for element in range(0, self.w):
            new_x = secrets.randbelow(self.p)
            while new_x == 0 or new_x in x:  # elements must be non-zero and distinct
                new_x = secrets.randbelow(self.p)
            x.append(new_x)
        self.x = x
        return self.x

    '''
        choose_a chooses t-1 elements of Z_p
        these are the random a_i values, 1 <= i <= t-1
    '''

    def choose_a(self):
        a = []
        for element in range(0, self.t - 1):
            a.append(secrets.randbelow(self.p))
        self.a = a
        return self.a

    '''
        generate_shares creates the secret shares to distribute
        these are the y_i values, 1 <= i <= w
    '''

    def generate_shares(self):
        y = []
        self.choose_a()
        for idx, element in enumerate(range(0, self.w)):
            poly = np.array(self.a + [self.k], dtype=object)
            ecs = np.array(self.x[idx], dtype=object)
            y.append(np.polyval(poly, ecs) % self.p)
        self.y = y
        return self.y

    '''
        KEY RECONSTRUCTION
    '''

    '''
        The following two functions are used to calculate the inverse modulo p
        of a number (we need it when we calculate the b values, to handle
        the denominator)
        
        Adapted from https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm
    '''

    def egcd(self, number1, number2):
        if number1 == 0:
            return (number2, 0, 1)
        else:
            g, y, x = self.egcd(number2 % number1, number1)
            return g, x - (number2 // number1) * y, y

    def modinv(self, number, m):
        g, x, y = self.egcd(number, m)
        if g != 1:
            raise Exception('modular inverse does not exist')
        else:
            return x % m

    '''
        calculate_b is needed to reconstruct the key from the shares
        Part of the Lagrange interpolation formula
        
        This is the simplified version that we can use when we want to reconstruct the key
        (since x=0 in this case)
    '''

    def calculate_b(self, x, x_values):
        acc = 1
        for element in x_values:
            if element != x:  # do not multiply the current x in the formula
                acc = acc * element
                if element - x >= 0:
                    inverse = self.modinv(element - x, self.p)
                else:
                    inverse = self.modinv(self.p - abs(element - x),
                                          self.p)  # https://math.stackexchange.com/questions/355066/find-the-inverse-modulo-of-a-number-got-a-negative-result
                acc = acc * inverse
        return acc % self.p

    '''
        calculate_b_full is the full Lagrange interpolation formula
        
        This is the formula we need when we want to calculate the share for a specific x != 0
        In this case, since x is not zero, we need to use the full formula
    '''

    def calculate_b_full(self, root, x, x_values):
        acc = 1
        for element in x_values:
            if element != x:  # do not multiply the current x in the formula
                acc = acc * (root-element)
                if element - x >= 0:
                    inverse = self.modinv(element - x, self.p)
                else:
                    inverse = self.modinv(self.p - abs(element - x),
                                          self.p)  # https://math.stackexchange.com/questions/355066/find-the-inverse-modulo-of-a-number-got-a-negative-result
                acc = acc * inverse
        return acc % self.p

    '''
        Take the values of x and the values of y as input
        (i.e. the coordinates of the points on the plane)
        and reconstruct the key by summing the products of b_i * y_i mod p
    '''

    def reconstruct_key(self, x, y):
        if len(y) < self.t:
            print("Not enough shares. Key reconstruction won't be possible.")
        else:
            b = []
            k = 0
            for index in range(len(y)):
                b.append(self.calculate_b(x[index], x))
                k = k + b[index] * y[index]
            return k % self.p

    def calculate_y(self, x, x_values, y_values):
        b = []
        y = 0
        for index in range(len(y_values)):
            b.append(self.calculate_b_full(x, x_values[index], x_values))
            y = y + b[index] * y_values[index]
        return y % self.p

    def validate_shares(self, x_values, y_values):
        comb_x = list(itertools.combinations(x_values, self.t))
        comb_y = list(itertools.combinations(y_values, self.t))
        key = sss.reconstruct_key(comb_x[0], comb_y[0])
        for index, element in enumerate(comb_x):
            next_key = sss.reconstruct_key(comb_x[index], comb_y[index])
            if (not(next_key == key)):
                return False
                exit()
        return True

    def find_defective_share(self, x_values, y_values):
        comb_x = list(itertools.combinations(x_values, self.t))
        comb_y = list(itertools.combinations(y_values, self.t))
        array_of_keys = []
        for index, element in enumerate(comb_x):
            key = sss.reconstruct_key(comb_x[index], comb_y[index])
            array_of_keys.append([comb_x[index], key])

        # Count how many times a key appears
        # The one with most occurrences is likely the correct key
        array_count = []
        for element in array_of_keys:
            array_count.append([element[1], sum(x.count(element[1]) for x in array_of_keys)])

        # Sort the array to find the most common key
        sorted_array = sorted(array_count, key=lambda x: x[1], reverse=True)
        likely_key = sorted_array[0][0]

        # Find the defective share, i.e. the intersection of every share returning the wrong result
        array_of_x = []
        for element in array_of_keys:
            if element[1] != likely_key: # only consider shares with the wrong key
                array_of_x.append(element[0])
        result = set(array_of_x[0]).intersection(*array_of_x)
        return list(result)[0]


'''
    Generate shares from a key
'''
sss = SSS(13, 17, 5, 3)
print("x = " + str(sss.choose_x()))
print("y = " + str(sss.generate_shares()))
# manual test
print("The key is " + str(sss.reconstruct_key([1, 3, 5], [8, 10, 11])))  # must return 13
# Calculate the share for different x
print("The share for x=1 is " + str(sss.calculate_y(1, [0,3,5], [sss.k, 10, 11])))
print("The share for x=3 is " + str(sss.calculate_y(3, [0,1,5], [sss.k, 8, 11])))
print("The share for x=5 is " + str(sss.calculate_y(5, [0,1,3], [sss.k, 8, 10])))

sss = SSS(1234, 1613, 6, 3)
print("x = " + str(sss.choose_x()))
print("y = " + str(sss.generate_shares()))
# manual test
print("The key is " + str(sss.reconstruct_key([1, 2, 3], [1494, 329, 965])))  # must return 1234
# Calculate the share for different x
print("The share for x=1 is " + str(sss.calculate_y(1, [0,2,3], [sss.k, 329, 965])))
print("The share for x=2 is " + str(sss.calculate_y(2, [0,1,3], [sss.k, 1494, 965])))
print("The share for x=3 is " + str(sss.calculate_y(3, [0,1,2], [sss.k, 1494, 329])))

sss = SSS(31318, 31847, 10, 5)
print("x = " + str(sss.choose_x()))
print("y = " + str(sss.generate_shares()))
# manual test
print(sss.reconstruct_key([413, 432, 451, 470, 489], [25439, 14847, 24780, 5910, 12734]))
print(sss.reconstruct_key([584, 432, 451, 470, 489], [21462, 14847, 24780, 5910, 12734]))
print(sss.reconstruct_key([584, 413, 565, 546, 489], [21462, 25439, 20806, 28578, 12734]))
print(sss.reconstruct_key([489, 565, 451, 470, 527], [12734, 20806, 24780, 5910, 12555]))
print(sss.reconstruct_key([508, 432, 584, 470, 489], [12492, 14847, 21462, 5910, 12734]))
# Calculate the share for different x
print("The share for x=413 is " + str(sss.calculate_y(413, [0, 432, 451, 470, 489], [sss.k, 14847, 24780, 5910, 12734])))
print("The share for x=584 is " + str(sss.calculate_y(584, [413, 432, 451, 470, 489], [25439, 14847, 24780, 5910, 12734])))
print("The share for x=508 is " + str(sss.calculate_y(508, [489, 565, 451, 470, 527], [12734, 20806, 24780, 5910, 12555])))
print("The share for x=0 is " + str(sss.calculate_y(0, [413, 432, 451, 470, 489], [25439, 14847, 24780, 5910, 12734])))

'''
    Share verification
'''

sss = SSS(1234, 94875355691, 9, 5)
x = [11, 22, 33, 44, 55, 66, 77, 88, 99]
y = [537048626, 89894377870, 65321160237, 18374404957, 24564576435, 87371334299, 60461341922, 10096524973, 81367619987]
print(sss.reconstruct_key([11, 22, 33, 44, 55],[537048626, 89894377870, 65321160237, 18374404957, 24564576435]))
print(sss.reconstruct_key([22, 33, 44, 55, 66],[89894377870, 65321160237, 18374404957, 24564576435, 87371334299]))
print(sss.reconstruct_key([33, 44, 55, 66, 77],[65321160237, 18374404957, 24564576435, 87371334299, 60461341922]))
print(sss.reconstruct_key([44, 55, 66, 77, 88],[18374404957, 24564576435, 87371334299, 60461341922, 10096524973]))
print(sss.reconstruct_key([55, 66, 77, 88, 99],[24564576435, 87371334299, 60461341922, 10096524973, 81367619987]))
print(sss.reconstruct_key([11, 22, 33, 44, 55, 66, 77, 88, 99], [537048626, 89894377870, 65321160237, 18374404957, 24564576435, 87371334299, 60461341922, 10096524973, 81367619987]))
print(sss.reconstruct_key([22, 33, 44, 55, 66, 77, 88, 99], [89894377870, 65321160237, 18374404957, 24564576435, 87371334299, 60461341922, 10096524973, 81367619987]))
print(sss.reconstruct_key([11, 22, 33, 44, 55, 66, 77, 88], [537048626, 89894377870, 65321160237, 18374404957, 24564576435, 87371334299, 60461341922, 10096524973]))

if sss.validate_shares(x, y):
    print("Shares are valid")
else:
    print("Shares are not valid")
    print("The defective share is " + str(sss.find_defective_share([11, 22, 33, 44, 55, 66, 77, 88, 99], [537048626, 89894377870, 65321160237, 18374404957, 24564576435, 87371334299, 60461341922, 10096524973, 81367619987])))

x = [16, 9, 3, 13, 5]
y = [8, 5, 13, 12, 9]
The key is 13
The share for x=1 is 8
The share for x=3 is 10
The share for x=5 is 11
x = [275, 1103, 923, 1319, 879, 1138]
y = [1204, 277, 953, 95, 1516, 276]
The key is 1234
The share for x=1 is 1494
The share for x=2 is 329
The share for x=3 is 965
x = [13040, 10282, 6502, 30648, 21545, 22106, 26659, 15062, 23978, 7142]
y = [12213, 20805, 23842, 15946, 6055, 23501, 18796, 18885, 28152, 16972]
31318
31318
31318
31318
31318
The share for x=413 is 25439
The share for x=584 is 21462
The share for x=508 is 12492
The share for x=0 is 31318
69147041214
73851808527
45623204649
16614591340
45623204649
45623204649
12382771831
20846410849
Shares are not valid
The defective share is 55


In [ ]:
import random
from math import ceil
from decimal import Decimal

FIELD_SIZE = 10**5


def reconstruct_secret(shares):
	"""
	Combines individual shares (points on graph)
	using Lagranges interpolation.

	`shares` is a list of points (x, y) belonging to a
	polynomial with a constant of our key.
	"""
	sums = 0
	prod_arr = []

	for j, share_j in enumerate(shares):
		xj, yj = share_j
		prod = Decimal(1)

		for i, share_i in enumerate(shares):
			xi, _ = share_i
			if i != j:
				prod *= Decimal(Decimal(xi)/(xi-xj))

		prod *= yj
		sums += Decimal(prod)

	return int(round(Decimal(sums), 0))


def polynom(x, coefficients):
	"""
	This generates a single point on the graph of given polynomial
	in `x`. The polynomial is given by the list of `coefficients`.
	"""
	point = 0
	# Loop through reversed list, so that indices from enumerate match the
	# actual coefficient indices
	for coefficient_index, coefficient_value in enumerate(coefficients[::-1]):
		point += x ** coefficient_index * coefficient_value
	return point


def coeff(t, secret):
	"""
	Randomly generate a list of coefficients for a polynomial with
	degree of `t` - 1, whose constant is `secret`.

	For example with a 3rd degree coefficient like this:
		3x^3 + 4x^2 + 18x + 554

		554 is the secret, and the polynomial degree + 1 is
		how many points are needed to recover this secret.
		(in this case it's 4 points).
	"""
	coeff = [random.randrange(0, FIELD_SIZE) for _ in range(t - 1)]
	coeff.append(secret)
	return coeff


def generate_shares(n, m, secret):
	"""
	Split given `secret` into `n` shares with minimum threshold
	of `m` shares to recover this `secret`, using SSS algorithm.
	"""
	coefficients = coeff(m, secret)
	shares = []

	for i in range(1, n+1):
		x = random.randrange(1, FIELD_SIZE)
		shares.append((x, polynom(x, coefficients)))

	return shares



	# (3,5) sharing scheme
t, n = 3, 5
secret = 1234
print(f'Original Secret: {secret}')

	# Phase I: Generation of shares
shares = generate_shares(n, t, secret)
print(f'Shares: {", ".join(str(share) for share in shares)}')

	# Phase II: Secret Reconstruction
	# Picking t shares randomly for
	# reconstruction
pool = random.sample(shares, t)
print(f'Combining shares: {", ".join(str(share) for share in pool)}')
print(f'Reconstructed secret: {reconstruct_secret(pool)}')



Original Secret: 1234
Shares: (91861, 452471707592907), (55651, 166064392269177), (79306, 337241912407692), (533, 15246643723), (21059, 23780021871961)
Combining shares: (55651, 166064392269177), (91861, 452471707592907), (21059, 23780021871961)
Reconstructed secret: 1234


In [ ]:
import random

def getAdditiveShares(secret, N, fieldSize):
	'''Generate N additive shares from 'secret' in finite field of size 'fieldSize'.'''

	# Generate n-1 shares randomly
	shares = [random.randrange(fieldSize) for i in range(N-1)]

	# Append final share by subtracting all shares from secret
	# Modulo is done with fieldSize to ensure share is within finite field
	shares.append((secret - sum(shares)) % fieldSize )
	return shares

def reconstructSecret(shares, fieldSize):
	'''Regenerate secret from additive shares'''
	return sum(shares) % fieldSize


shares = getAdditiveShares(1234, 5, 10**5)
print('Shares are:', shares)
	
	# Reconstructing the secret from shares
print('Reconstructed secret:', reconstructSecret(shares, 10**5))


Shares are: [42070, 7010, 26809, 52420, 72925]
Reconstructed secret: 1234


In [ ]:
# Additive Sharing with facility to Refresh shares via Proactivization
import random

def getAdditiveShares(secret, N, fieldSize):
	'''Generate N additive shares from 'secret' in finite field of size 'fieldSize'.'''

	# Generate n-1 shares randomly
	shares = [random.randrange(fieldSize) for i in range(N-1)]
	# Append final share by subtracting all shares from secret
	shares.append((secret - sum(shares)) % fieldSize )
	return shares

def reconstructSecret(shares, fieldSize):
	'''Regenerate secret from additive shares'''
	return sum(shares) % fieldSize

def proactivizeShares(shares):
	'''Refreshed shares by proactivization'''
	
	n = len(shares)
	refreshedShares = [0]*n

	for s in shares:

		# Divide each share into sub-fragments using additive sharing
		subShares = getAdditiveShares(s, n, 10**5)

		# Add subfragments of corresponding parties
		for p, sub in enumerate(subShares):
			refreshedShares[p] += sub
			
	return refreshedShares


shares = getAdditiveShares(1234, 5, 10**5)
print('Shares are:', shares)

	# Running Proactivization
newShares = proactivizeShares(shares)
print('Refreshed Shares are:', newShares)
	
	# Reconstructing secret from refreshed shares
print('Secret:', reconstructSecret(newShares, 10**5))


Shares are: [6213, 84782, 82019, 73730, 54490]
Refreshed Shares are: [317756, 353853, 313948, 272586, 343091]
Secret: 1234


In [ ]:
from joblib.numpy_pickle_utils import xrange
from numpy import *


class NeuralNet(object):
	def __init__(self):
		# Generate random numbers
		random.seed(1)

		# Assign random weights to a 3 x 1 matrix,
		self.synaptic_weights = 2 * random.random((3, 1)) - 1

	# The Sigmoid function
	def __sigmoid(self, x):
		return 1 / (1 + exp(-x))

	# The derivative of the Sigmoid function.
	# This is the gradient of the Sigmoid curve.
	def __sigmoid_derivative(self, x):
		return x * (1 - x)

	# Train the neural network and adjust the weights each time.
	def train(self, inputs, outputs, training_iterations):
		for iteration in xrange(training_iterations):
			# Pass the training set through the network.
			output = self.learn(inputs)

			# Calculate the error
			error = outputs - output

			# Adjust the weights by a factor
			factor = dot(inputs.T, error * self.__sigmoid_derivative(output))
			self.synaptic_weights += factor

		# The neural network thinks.

	def learn(self, inputs):
		return self.__sigmoid(dot(inputs, self.synaptic_weights))


neural_network = NeuralNet()

	# The training set.
inputs = array([[0, 1, 1], [1, 0, 0], [1, 0, 1]])
outputs = array([[1, 0, 1]]).T

	# Train the neural network
neural_network.train(inputs, outputs, 10000)

	# Test the neural network with a test example.
print(neural_network.learn(array([1, 0, 1])))


[0.9897704]
